In [20]:
%%capture
!pip install unsloth --no-cache-dir
!pip install triton==3.2.0 --no-cache-dir

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
from unsloth import FastLanguageModel
import triton
print(triton.__version__)
import torch

hf_token = "your_hf_token_here"

max_seq_length=2048
dtype = None # auto detection
load_in_4bit = True

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
dataPath = "data/reddit/reddit_linux_qna_data.csv"
outputPath = "./model/results"
model_savePath  = "./model/tinyllama-linux-qna-lora"

3.2.0


# Define model & tokenizer

In [22]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
    # token=hf_token
)

==((====))==  Unsloth 2025.5.8: Fast Llama patching. Transformers: 4.52.3.
   \\   /|    NVIDIA GeForce RTX 3050 Ti Laptop GPU. Num GPUs = 1. Max memory: 3.691 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


# Configure LoRA

In [23]:
# Configure LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r=32,  # LoRA rank
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,  # Rank stabilized LoRA
    loftq_config=None  # No LoftQ for simplicity
)

# Data Prep

In [24]:
prompt = """Below is a question relating to the linux operating system, paired with a paragraph describing further context. Write a short, simple, consise and comprehensive response to the question.
### Question
{}
### Context
{}
### Response
{}"""
EOS_TOKEN = tokenizer.eos_token
def format_prompt_func(examples):
    questions = examples["question"]
    contexts = examples["context"]
    responses = examples["response"]
    
    texts = []
    for q, c, r in zip(questions, contexts, responses):
        text = prompt.format(q, c, r)
        texts.append(text)
    return {
        "text": texts
    }

from datasets import load_dataset

dataset = load_dataset("csv", data_files=dataPath, split="train")
dataset = dataset.map(format_prompt_func, batched=True)

In [25]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForLanguageModeling
from unsloth import is_bfloat16_supported

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal language modeling
    mlm_probability=0.0,  # Explicitly set to avoid NoneType
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = True, # pack short sequences together to save time
    data_collator=data_collator,
    args = TrainingArguments(
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 8,
        warmup_ratio = 0.1,
        num_train_epochs = 3,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine", # "cosine" or "linear"
        seed= 3407,
        output_dir = outputPath,
        report_to = "none"
    )
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/1344 [00:00<?, ? examples/s]

In [26]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,344 | Num Epochs = 3 | Total steps = 504
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 25,231,360/4,000,000,000 (0.63% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.714600
2,2.858700
3,2.955000
4,3.003200
5,2.564800
6,2.877300
7,2.923300
8,2.771100
9,2.855000
10,2.631300


TrainOutput(global_step=504, training_loss=2.23348092441521, metrics={'train_runtime': 1836.7453, 'train_samples_per_second': 2.195, 'train_steps_per_second': 0.274, 'total_flos': 9331774889103360.0, 'train_loss': 2.23348092441521})

# Save model

In [27]:
model.save_pretrained(model_savePath)
tokenizer.save_pretrained(model_savePath)

('./model/tinyllama-linux-qna-lora/tokenizer_config.json',
 './model/tinyllama-linux-qna-lora/special_tokens_map.json',
 './model/tinyllama-linux-qna-lora/chat_template.jinja',
 './model/tinyllama-linux-qna-lora/tokenizer.model',
 './model/tinyllama-linux-qna-lora/added_tokens.json',
 './model/tinyllama-linux-qna-lora/tokenizer.json')